In [ ]:
pip install ta

In [ ]:
import pandas as pd
import numpy as np
import requests
import hashlib
import matplotlib.pyplot as plt
from ta import trend, momentum, volatility
from datetime import datetime, timedelta

# =========================
# Configuration and Settings
# =========================

# List of cryptocurrencies to analyze
CRYPTOCURRENCIES = ['bitcoin', 'ethereum', 'litecoin']  # CoinGecko IDs

# Fiat currencies for exchange rates
FIAT_CURRENCIES = ['USD', 'EUR', 'JPY', 'GBP', 'AUD']

# Timeframe for historical data
DAYS = 90  # Last 90 days

# Technical indicators settings
SMA_SHORT = 20  # Short-term Simple Moving Average
SMA_LONG = 50   # Long-term Simple Moving Average
RSI_PERIOD = 14
VOLATILITY_PERIOD = 14

# =========================
# Utility Functions
# =========================

def get_hash(data):
    """
    Generates a SHA-256 hash for the given data.
    """
    hash_object = hashlib.sha256(data.encode())
    return hash_object.hexdigest()

def fetch_coin_gecko_data(crypto_id, vs_currency='usd', days=90):
    """
    Fetches historical market data for a cryptocurrency from CoinGecko.
    """
    url = f'https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart'
    params = {
        'vs_currency': vs_currency,
        'days': days,
        'interval': 'daily'
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Error fetching data from CoinGecko for {crypto_id}")
    data = response.json()
    prices = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    prices['Date'] = pd.to_datetime(prices['timestamp'], unit='ms')
    prices.set_index('Date', inplace=True)
    prices['Close'] = prices['price']
    prices = prices[['Close']].sort_index()
    # Generate hash for data integrity
    prices['Hash'] = prices['Close'].apply(lambda x: get_hash(str(x)))
    return prices

def fetch_binance_data(symbol='BTCUSDT', interval='1d', limit=90):
    """
    Fetches historical klines data for a cryptocurrency from Binance.
    """
    url = 'https://api.binance.com/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': limit
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Error fetching data from Binance for {symbol}")
    data = response.json()
    klines = pd.DataFrame(data, columns=[
        'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume',
        'Close Time', 'Quote Asset Volume', 'Number of Trades',
        'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
    ])
    klines['Date'] = pd.to_datetime(klines['Close Time'], unit='ms')
    klines.set_index('Date', inplace=True)
    klines['Close'] = klines['Close'].astype(float)
    klines = klines[['Close']].sort_index()
    # Generate hash for data integrity
    klines['Hash'] = klines['Close'].apply(lambda x: get_hash(str(x)))
    return klines

def fetch_exchange_rates(base='USD'):
    """
    Fetches current exchange rates from multiple sources.
    """
    rates = {}
    # Source 1: ExchangeRate-API
    try:
        api_key = 'YOUR_EXCHANGERATE_API_KEY'  # Replace with your API key
        url = f'https://v6.exchangerate-api.com/v6/{api_key}/latest/{base}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            rates.update(data.get('conversion_rates', {}))
    except Exception as e:
        print(f"ExchangeRate-API failed: {e}")

    # Source 2: exchangerate.host
    try:
        url = f'https://api.exchangerate.host/latest?base={base}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            rates.update(data.get('rates', {}))
    except Exception as e:
        print(f"exchangerate.host failed: {e}")

    # Filter for desired fiat currencies
    filtered_rates = {currency: rates.get(currency) for currency in FIAT_CURRENCIES if rates.get(currency)}
    exchange_rates = pd.Series(filtered_rates, name='Exchange Rate')
    # Generate hash for data integrity
    exchange_rates_hash = get_hash(exchange_rates.to_json())
    return exchange_rates, exchange_rates_hash

# =========================
# Data Fetching
# =========================

def fetch_all_crypto_data():
    """
    Fetches historical data from multiple sources for all cryptocurrencies.
    """
    crypto_data = {}
    for crypto in CRYPTOCURRENCIES:
        try:
            # Fetch from CoinGecko
            cg_data = fetch_coin_gecko_data(crypto)
            # Fetch from Binance
            symbol = crypto_symbol_mapping(crypto)
            bn_data = fetch_binance_data(symbol=symbol)
            # Merge data from both sources
            combined = pd.concat([cg_data, bn_data], axis=1, join='inner', lsuffix='_cg', rsuffix='_bn')
            # Calculate average price from both sources
            combined['Close'] = combined[['Close_cg', 'Close_bn']].mean(axis=1)
            combined = combined[['Close']]
            # Generate a unique identifier for the crypto data
            combined['Data ID'] = combined.index.to_series().apply(lambda x: get_hash(x.strftime('%Y-%m-%d')))
            crypto_data[crypto] = combined
            print(f"Fetched data for {crypto}")
        except Exception as e:
            print(f"Failed to fetch data for {crypto}: {e}")
    return crypto_data

def crypto_symbol_mapping(crypto_id):
    """
    Maps CoinGecko crypto IDs to Binance symbols.
    """
    mapping = {
        'bitcoin': 'BTCUSDT',
        'ethereum': 'ETHUSDT',
        'litecoin': 'LTCUSDT',
        # Add more mappings as needed
    }
    return mapping.get(crypto_id, 'BTCUSDT')  # Default to BTCUSDT

# =========================
# Data Analysis
# =========================

def perform_technical_analysis(df):
    """
    Applies technical indicators to the DataFrame using the `ta` library.
    """
    # Simple Moving Averages
    df['SMA_Short'] = trend.sma_indicator(close=df['Close'], window=SMA_SHORT)
    df['SMA_Long'] = trend.sma_indicator(close=df['Close'], window=SMA_LONG)

    # Relative Strength Index
    df['RSI'] = momentum.rsi(close=df['Close'], window=RSI_PERIOD)

    # Bollinger Bands
    bollinger = trend.BollingerBands(close=df['Close'], window=20, window_dev=2)
    df['Bollinger_High'] = bollinger.bollinger_hband()
    df['Bollinger_Low'] = bollinger.bollinger_lband()

    # Average True Range (Volatility)
    df['ATR'] = volatility.average_true_range(high=df['Close'], low=df['Close'], close=df['Close'], window=VOLATILITY_PERIOD)

    return df

def identify_trends(df):
    """
    Identifies upward and downward trends based on technical indicators.
    """
    df['Trend'] = 'Neutral'
    # SMA Crossover Strategy
    df.loc[df['SMA_Short'] > df['SMA_Long'], 'Trend'] = 'Uptrend'
    df.loc[df['SMA_Short'] < df['SMA_Long'], 'Trend'] = 'Downtrend'

    # RSI Overbought/Oversold
    df['RSI_Status'] = 'Neutral'
    df.loc[df['RSI'] > 70, 'RSI_Status'] = 'Overbought'
    df.loc[df['RSI'] < 30, 'RSI_Status'] = 'Oversold'

    return df

def generate_signals(df):
    """
    Generates buy/sell signals based on trends and RSI.
    """
    df['Signal'] = None
    # Buy Signal: Uptrend and RSI oversold
    df.loc[(df['Trend'] == 'Uptrend') & (df['RSI_Status'] == 'Oversold'), 'Signal'] = 'Buy'
    # Sell Signal: Downtrend and RSI overbought
    df.loc[(df['Trend'] == 'Downtrend') & (df['RSI_Status'] == 'Overbought'), 'Signal'] = 'Sell'
    return df

# =========================
# Visualization
# =========================

def plot_crypto_analysis(df, crypto_name):
    """
    Plots the price and technical indicators for the cryptocurrency.
    """
    plt.figure(figsize=(14, 7))
    plt.plot(df.index, df['Close'], label='Close Price', color='blue')
    plt.plot(df.index, df['SMA_Short'], label=f'SMA {SMA_SHORT}', color='red')
    plt.plot(df.index, df['SMA_Long'], label=f'SMA {SMA_LONG}', color='green')

    # Plot Buy/Sell Signals
    buy_signals = df[df['Signal'] == 'Buy']
    sell_signals = df[df['Signal'] == 'Sell']
    plt.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='g', label='Buy Signal', s=100)
    plt.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='r', label='Sell Signal', s=100)

    plt.title(f'{crypto_name.capitalize()} Price and Technical Indicators')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.grid(True)
    plt.show()

# =========================
# Main Execution
# =========================

def main():
    # Step 1: Fetch cryptocurrency data from multiple sources
    crypto_data = fetch_all_crypto_data()

    # Step 2: Fetch exchange rates
    exchange_rates, exchange_rates_hash = fetch_exchange_rates(base='USD')
    print("\nCurrent Exchange Rates (Base: USD):")
    print(exchange_rates)
    print(f"Exchange Rates Data Hash: {exchange_rates_hash}\n")

    # Step 3: Perform technical analysis and identify trends
    for crypto, df in crypto_data.items():
        df = perform_technical_analysis(df)
        df = identify_trends(df)
        df = generate_signals(df)
        crypto_data[crypto] = df  # Update the dictionary with analyzed data
        print(f"Completed analysis for {crypto}")

        # Step 4: Visualization
        plot_crypto_analysis(df, crypto)

        # Display signals
        signals = df[df['Signal'].notnull()]
        print(f"Signals for {crypto}:")
        print(signals[['Close', 'Trend', 'RSI', 'RSI_Status', 'Signal']])
        print("\n" + "-"*50 + "\n")

if __name__ == "__main__":
    main()

In [27]:
def fetch_exchange_rates(base='USD'):
    """
    Fetches current exchange rates from exchangerate.host.
    """
    try:
        url = f'https://api.exchangerate.host/latest?base={base}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            rates = data.get('rates', {})
    except Exception as e:
        print(f"exchangerate.host failed: {e}")
        rates = {}

    # Filter for desired fiat currencies
    filtered_rates = {currency: rates.get(currency) for currency in FIAT_CURRENCIES if rates.get(currency)}
    exchange_rates = pd.Series(filtered_rates, name='Exchange Rate')
    # Generate hash for data integrity
    exchange_rates_hash = get_hash(exchange_rates.to_json())
    return exchange_rates, exchange_rates_hash

In [29]:
pip install pandas numpy requests matplotlib ta

In [31]:
!python crypto_analysis.py

#python crypto_analysis.py

python: can't open file 'C:\\Users\\todd-\\crypto_analysis.py': [Errno 2] No such file or directory


In [33]:
# Create a virtual environment named 'crypto_env'
python -m venv crypto_env

# Activate the virtual environment
# On Windows:
crypto_env\Scripts\activate
# On macOS/Linux:
source crypto_env/bin/activate

# Install dependencies within the virtual environment
pip install pandas numpy requests matplotlib ta

SyntaxError: invalid syntax (1845414933.py, line 2)

In [ ]:
import requests
import pandas as pd
import numpy as np
import talib as ta
import hashlib
import matplotlib.pyplot as plt

# Function to get crypto data (e.g., Bitcoin) from CoinGecko API
def get_crypto_data(symbol, days):
    url = f'https://api.coingecko.com/api/v3/coins/{symbol}/market_chart?vs_currency=usd&days={days}'
    response = requests.get(url)
    data = response.json()

    # Convert data to pandas DataFrame
    prices = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    prices['timestamp'] = pd.to_datetime(prices['timestamp'], unit='ms')
    return prices

# Hash function for analysis
def hash_price(price):
    return hashlib.sha256(str(price).encode()).hexdigest()

# Perform basic technical analysis
def perform_ta(prices):
    # Moving averages
    prices['SMA_50'] = ta.SMA(prices['price'], timeperiod=50)
    prices['SMA_200'] = ta.SMA(prices['price'], timeperiod=200)
    
    # RSI (Relative Strength Index)
    prices['RSI'] = ta.RSI(prices['price'], timeperiod=14)
    
    return prices

# Analyze the upward and downward ranges
def analyze_trends(prices):
    upward_trend = prices[prices['SMA_50'] > prices['SMA_200']]
    downward_trend = prices[prices['SMA_50'] < prices['SMA_200']]
    
    upward_hashes = upward_trend['price'].apply(hash_price)
    downward_hashes = downward_trend['price'].apply(hash_price)
    
    return upward_trend, downward_trend, upward_hashes, downward_hashes

# Main function to fetch data, analyze, and plot
def main():
    symbol = 'bitcoin'
    days = 365  # Fetch 1 year of data
    prices = get_crypto_data(symbol, days)
    
    # Perform technical analysis
    prices_ta = perform_ta(prices)
    
    # Analyze trends
    upward_trend, downward_trend, upward_hashes, downward_hashes = analyze_trends(prices_ta)
    
    # Plot data
    plt.figure(figsize=(10, 5))
    plt.plot(prices_ta['timestamp'], prices_ta['price'], label='Price')
    plt.plot(prices_ta['timestamp'], prices_ta['SMA_50'], label='SMA 50', color='orange')
    plt.plot(prices_ta['timestamp'], prices_ta['SMA_200'], label='SMA 200', color='red')
    plt.title(f'Bitcoin Price with Technical Indicators')
    plt.legend()
    plt.show()

    # Print sample hash analysis
    print("Sample Upward Trend Hashes:")
    print(upward_hashes.head())
    print("\nSample Downward Trend Hashes:")
    print(downward_hashes.head())

# Execute main function
if __name__ == "__main__":
    main()

In [ ]:
import requests
import pandas as pd
import numpy as np
import talib as ta
import hashlib
import matplotlib.pyplot as plt

# Function to get crypto data (e.g., Bitcoin) from CoinGecko API
def get_crypto_data(symbol, days):
    url = f'https://api.coingecko.com/api/v3/coins/{symbol}/market_chart?vs_currency=usd&days={days}'
    response = requests.get(url)
    data = response.json()

    # Convert data to pandas DataFrame
    prices = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    prices['timestamp'] = pd.to_datetime(prices['timestamp'], unit='ms')
    return prices

# Hash function for analysis
def hash_price(price):
    return hashlib.sha256(str(price).encode()).hexdigest()

# Perform basic technical analysis
def perform_ta(prices):
    # Moving averages
    prices['SMA_50'] = ta.SMA(prices['price'], timeperiod=50)
    prices['SMA_200'] = ta.SMA(prices['price'], timeperiod=200)
    
    # RSI (Relative Strength Index)
    prices['RSI'] = ta.RSI(prices['price'], timeperiod=14)
    
    return prices

# Analyze the upward and downward ranges
def analyze_trends(prices):
    upward_trend = prices[prices['SMA_50'] > prices['SMA_200']]
    downward_trend = prices[prices['SMA_50'] < prices['SMA_200']]
    
    upward_hashes = upward_trend['price'].apply(hash_price)
    downward_hashes = downward_trend['price'].apply(hash_price)
    
    return upward_trend, downward_trend, upward_hashes, downward_hashes

# Main function to fetch data, analyze, and plot
def main():
    symbol = 'bitcoin'
    days = 365  # Fetch 1 year of data
    prices = get_crypto_data(symbol, days)
    
    # Perform technical analysis
    prices_ta = perform_ta(prices)
    
    # Analyze trends
    upward_trend, downward_trend, upward_hashes, downward_hashes = analyze_trends(prices_ta)
    
    # Plot data
    plt.figure(figsize=(10, 5))
    plt.plot(prices_ta['timestamp'], prices_ta['price'], label='Price')
    plt.plot(prices_ta['timestamp'], prices_ta['SMA_50'], label='SMA 50', color='orange')
    plt.plot(prices_ta['timestamp'], prices_ta['SMA_200'], label='SMA 200', color='red')
    plt.title(f'Bitcoin Price with Technical Indicators')
    plt.legend()
    plt.show()

    # Print sample hash analysis
    print("Sample Upward Trend Hashes:")
    print(upward_hashes.head())
    print("\nSample Downward Trend Hashes:")
    print(downward_hashes.head())

# Execute main function
if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import numpy as np
import requests
import hashlib
import matplotlib.pyplot as plt
from ta import trend, momentum, volatility
from datetime import datetime, timedelta

# =========================
# Configuration and Settings
# =========================

# List of cryptocurrencies to analyze
CRYPTOCURRENCIES = ['bitcoin', 'ethereum', 'litecoin']  # CoinGecko IDs

# Fiat currencies for exchange rates
FIAT_CURRENCIES = ['USD', 'EUR', 'JPY', 'GBP', 'AUD']

# Timeframe for historical data
DAYS = 90  # Last 90 days

# Technical indicators settings
SMA_SHORT = 20  # Short-term Simple Moving Average
SMA_LONG = 50   # Long-term Simple Moving Average
RSI_PERIOD = 14
VOLATILITY_PERIOD = 14

# =========================
# Utility Functions
# =========================

def get_hash(data):
    """
    Generates a SHA-256 hash for the given data.
    """
    hash_object = hashlib.sha256(data.encode())
    return hash_object.hexdigest()

def fetch_coin_gecko_data(crypto_id, vs_currency='usd', days=90):
    """
    Fetches historical market data for a cryptocurrency from CoinGecko.
    """
    url = f'https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart'
    params = {
        'vs_currency': vs_currency,
        'days': days,
        'interval': 'daily'
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Error fetching data from CoinGecko for {crypto_id}")
    data = response.json()
    prices = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    prices['Date'] = pd.to_datetime(prices['timestamp'], unit='ms')
    prices.set_index('Date', inplace=True)
    prices['Close'] = prices['price']
    prices = prices[['Close']].sort_index()
    # Generate hash for data integrity
    prices['Hash'] = prices['Close'].apply(lambda x: get_hash(str(x)))
    return prices

def fetch_binance_data(symbol='BTCUSDT', interval='1d', limit=90):
    """
    Fetches historical klines data for a cryptocurrency from Binance.
    """
    url = 'https://api.binance.com/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': limit
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Error fetching data from Binance for {symbol}")
    data = response.json()
    klines = pd.DataFrame(data, columns=[
        'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume',
        'Close Time', 'Quote Asset Volume', 'Number of Trades',
        'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
    ])
    klines['Date'] = pd.to_datetime(klines['Close Time'], unit='ms')
    klines.set_index('Date', inplace=True)
    klines['Close'] = klines['Close'].astype(float)
    klines = klines[['Close']].sort_index()
    # Generate hash for data integrity
    klines['Hash'] = klines['Close'].apply(lambda x: get_hash(str(x)))
    return klines

def fetch_exchange_rates(base='USD'):
    """
    Fetches current exchange rates from multiple sources.
    """
    rates = {}
    # Source 1: ExchangeRate-API
    try:
        api_key = 'YOUR_EXCHANGERATE_API_KEY'  # Replace with your API key
        url = f'https://v6.exchangerate-api.com/v6/{api_key}/latest/{base}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            rates.update(data.get('conversion_rates', {}))
    except Exception as e:
        print(f"ExchangeRate-API failed: {e}")

    # Source 2: exchangerate.host
    try:
        url = f'https://api.exchangerate.host/latest?base={base}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            rates.update(data.get('rates', {}))
    except Exception as e:
        print(f"exchangerate.host failed: {e}")

    # Filter for desired fiat currencies
    filtered_rates = {currency: rates.get(currency) for currency in FIAT_CURRENCIES if rates.get(currency)}
    exchange_rates = pd.Series(filtered_rates, name='Exchange Rate')
    # Generate hash for data integrity
    exchange_rates_hash = get_hash(exchange_rates.to_json())
    return exchange_rates, exchange_rates_hash

# =========================
# Data Fetching
# =========================

def fetch_all_crypto_data():
    """
    Fetches historical data from multiple sources for all cryptocurrencies.
    """
    crypto_data = {}
    for crypto in CRYPTOCURRENCIES:
        try:
            # Fetch from CoinGecko
            cg_data = fetch_coin_gecko_data(crypto)
            # Fetch from Binance
            symbol = crypto_symbol_mapping(crypto)
            bn_data = fetch_binance_data(symbol=symbol)
            # Merge data from both sources
            combined = pd.concat([cg_data, bn_data], axis=1, join='inner', lsuffix='_cg', rsuffix='_bn')
            # Calculate average price from both sources
            combined['Close'] = combined[['Close_cg', 'Close_bn']].mean(axis=1)
            combined = combined[['Close']]
            # Generate a unique identifier for the crypto data
            combined['Data ID'] = combined.index.to_series().apply(lambda x: get_hash(x.strftime('%Y-%m-%d')))
            crypto_data[crypto] = combined
            print(f"Fetched data for {crypto}")
        except Exception as e:
            print(f"Failed to fetch data for {crypto}: {e}")
    return crypto_data

def crypto_symbol_mapping(crypto_id):
    """
    Maps CoinGecko crypto IDs to Binance symbols.
    """
    mapping = {
        'bitcoin': 'BTCUSDT',
        'ethereum': 'ETHUSDT',
        'litecoin': 'LTCUSDT',
        # Add more mappings as needed
    }
    return mapping.get(crypto_id, 'BTCUSDT')  # Default to BTCUSDT

# =========================
# Data Analysis
# =========================

def perform_technical_analysis(df):
    """
    Applies technical indicators to the DataFrame.
    """
    # Simple Moving Averages
    df['SMA_Short'] = trend.SMAIndicator(close=df['Close'], window=SMA_SHORT).sma_indicator()
    df['SMA_Long'] = trend.SMAIndicator(close=df['Close'], window=SMA_LONG).sma_indicator()
    
    # Relative Strength Index
    df['RSI'] = momentum.RSIIndicator(close=df['Close'], window=RSI_PERIOD).rsi()
    
    # Bollinger Bands
    bollinger = trend.BollingerBands(close=df['Close'], window=20, window_dev=2)
    df['Bollinger_High'] = bollinger.bollinger_hband()
    df['Bollinger_Low'] = bollinger.bollinger_lband()
    
    # Average True Range (Volatility)
    df['ATR'] = volatility.AverageTrueRange(high=df['Close'], low=df['Close'], close=df['Close'], window=VOLATILITY_PERIOD).average_true_range()
    
    return df

def identify_trends(df):
    """
    Identifies upward and downward trends based on technical indicators.
    """
    df['Trend'] = 'Neutral'
    # SMA Crossover Strategy
    df.loc[df['SMA_Short'] > df['SMA_Long'], 'Trend'] = 'Uptrend'
    df.loc[df['SMA_Short'] < df['SMA_Long'], 'Trend'] = 'Downtrend'
    
    # RSI Overbought/Oversold
    df.loc[df['RSI'] > 70, 'RSI_Status'] = 'Overbought'
    df.loc[df['RSI'] < 30, 'RSI_Status'] = 'Oversold'
    df['RSI_Status'].fillna('Neutral', inplace=True)
    
    return df

def generate_signals(df):
    """
    Generates buy/sell signals based on trends and RSI.
    """
    df['Signal'] = None
    # Buy Signal: Uptrend and RSI oversold
    df.loc[(df['Trend'] == 'Uptrend') & (df['RSI_Status'] == 'Oversold'), 'Signal'] = 'Buy'
    # Sell Signal: Downtrend and RSI overbought
    df.loc[(df['Trend'] == 'Downtrend') & (df['RSI_Status'] == 'Overbought'), 'Signal'] = 'Sell'
    return df

# =========================
# Visualization
# =========================

def plot_crypto_analysis(df, crypto_name):
    """
    Plots the price and technical indicators for the cryptocurrency.
    """
    plt.figure(figsize=(14, 7))
    plt.plot(df.index, df['Close'], label='Close Price', color='blue')
    plt.plot(df.index, df['SMA_Short'], label=f'SMA {SMA_SHORT}', color='red')
    plt.plot(df.index, df['SMA_Long'], label=f'SMA {SMA_LONG}', color='green')
    
    # Plot Buy/Sell Signals
    buy_signals = df[df['Signal'] == 'Buy']
    sell_signals = df[df['Signal'] == 'Sell']
    plt.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='g', label='Buy Signal', s=100)
    plt.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='r', label='Sell Signal', s=100)
    
    plt.title(f'{crypto_name.capitalize()} Price and Technical Indicators')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.grid(True)
    plt.show()

# =========================
# Main Execution
# =========================

def main():
    # Step 1: Fetch cryptocurrency data from multiple sources
    crypto_data = fetch_all_crypto_data()
    
    # Step 2: Fetch exchange rates
    exchange_rates, exchange_rates_hash = fetch_exchange_rates(base='USD')
    print("\nCurrent Exchange Rates (Base: USD):")
    print(exchange_rates)
    print(f"Exchange Rates Data Hash: {exchange_rates_hash}\n")
    
    # Step 3: Perform technical analysis and identify trends
    for crypto, df in crypto_data.items():
        df = perform_technical_analysis(df)
        df = identify_trends(df)
        df = generate_signals(df)
        crypto_data[crypto] = df  # Update the dictionary with analyzed data
        print(f"Completed analysis for {crypto}")
        
        # Step 4: Visualization
        plot_crypto_analysis(df, crypto)
        
        # Display signals
        signals = df[df['Signal'].notnull()]
        print(f"Signals for {crypto}:")
        print(signals[['Close', 'Trend', 'RSI', 'RSI_Status', 'Signal']])
        print("\n" + "-"*50 + "\n")

if __name__ == "__main__":
    main()

In [ ]:
python crypto_analysis.py